In [2]:
from glob import glob
from tqdm import tqdm
#from rdkit import Chem
import numpy as np
import pandas as pd
import re
import shutil
import os

import autogluon
from autogluon.common.utils.utils import setup_outputdir
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.core.utils.loaders import load_pkl
from autogluon.core.utils.savers import save_pkl
import os.path
import sklearn
from sklearn.model_selection import train_test_split
import autogluon.core as ag
import pandas as pd
import numpy as np

import pandas as pd
import random
import argparse
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import ray
import glob

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/opt/conda/envs/bigcon/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import datetime 
import time     

### K fold ( Set 5 Fold)

In [4]:
def k_fold_result(data):
    ## Test set setting
    data = data.reset_index().iloc[:, 1:]
    
    train, test = train_test_split(data, test_size=0.15,   random_state=20205289,  stratify=data['is_applied'])

    train = train.reset_index().iloc[:, 1:]
    test = test.reset_index().iloc[:, 1:]

    from sklearn.model_selection import StratifiedKFold
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=20205289)
    for fold, (train_idx, val_idx) in enumerate(kf.split(range(len(train)), y=train['is_applied'])):
        train.loc[val_idx, 'fold'] = fold
        
    return train, test

### Modeling

In [5]:
def modeling(dataset, fold):
    
    dataset, test = k_fold_result(dataset)
    test.to_csv(f"./fold_data/model_all_test.csv")
    
    train = dataset.reset_index().iloc[:, 1:]
    train["is_applied"] = train["is_applied"].astype(int)
    
    for i in range(0, fold):

        val_idx = list(train[train['fold'] == int(i)].index)
        val_df = train[train['fold'] == i].reset_index(drop=True).iloc[:, :-1]
        train_df = train[train['fold'] != i].reset_index(drop=True).iloc[:, :-1]

        labels = 'is_applied'
        problem_types = 'binary'  
        save_path = f'model_test_all/model_all_fold_{i}'  

        excluded_model_types = ['KNN', "NN_TORCH", "FASTAI", "CAT", "Cat"]
        multi_predictor = TabularPredictor(label=labels, problem_type=problem_types, eval_metric="f1", path=save_path)
        multi_predictor.fit(train_df, tuning_data=val_df, excluded_model_types=excluded_model_types, hyperparameters = "light") #['default', 'light', 'very_light', 'toy', 'multimodal', 'interpretable']
        
        train_df.to_csv(f"./fold_data/model_all_fold_{i}.csv")
        val_df.to_csv(f"./fold_data/model_all_fold_{i}.csv")
        
        time.sleep(60)
        print("TIME")
    
    return test
    

In [6]:
full_data = pd.read_csv("./dataset/full_data.csv")

# le_income_type

le_purpose_type = LabelEncoder()
le_purpose_type = le_purpose_type.fit(full_data["purpose"])
le_purpose_fitted = le_purpose_type.fit_transform(full_data["purpose"])
purpose_inverted = le_purpose_type.inverse_transform(le_purpose_fitted)

print(le_purpose_type.classes_)

full_data["purpose"] = le_purpose_fitted
full_data["bank_id"] = full_data["bank_id"].astype('category')
full_data["product_id"] = full_data["product_id"].astype('category')
full_data["loan_limit"] = full_data["loan_limit"].astype('category')
full_data["birth_year"] = full_data["birth_year"].astype('category')

full_data["gender"] = full_data["gender"].astype('category')
full_data["credit_score"] = full_data["credit_score"].astype('category')
full_data["yearly_income"] = full_data["yearly_income"].astype('category')
full_data["income_type"] = full_data["income_type"].astype('category')
full_data["company_enter_month"] = full_data["company_enter_month"].astype('category')

full_data["employment_type"] = full_data["employment_type"].astype('category')
full_data["houseown_type"] = full_data["houseown_type"].astype('category')
full_data["desired_amount"] = full_data["desired_amount"].astype('category')
full_data["existing_loan_cnt"] = full_data["existing_loan_cnt"].astype('category')
full_data["existing_loan_amt"] = full_data["existing_loan_amt"].astype('category')

full_data["credit_score_nan"] = full_data["credit_score_nan"].astype('category')
full_data["gender_birth_nan"] = full_data["gender_birth_nan"].astype('category')
full_data["loan_limit/max_loan_limit_binary"] = full_data["loan_limit/max_loan_limit_binary"].astype('category')
full_data["loan_rate/min_loan_rate_binary"] = full_data["loan_rate/min_loan_rate_binary"].astype('category')

full_data["desired_amount/existing_loan_amt_binary"] = full_data["desired_amount/existing_loan_amt_binary"].astype('category')
full_data["hour"] = full_data["hour"].astype('category')
full_data["dayofweek"] = full_data["dayofweek"].astype('category')
full_data["purpose"] = full_data["purpose"].astype('category')
full_data["existing_loan_cnt_binary"] = full_data["existing_loan_cnt_binary"].astype("category")

full_data = full_data.drop(["application_id", "user_id"], axis = 1)

['기타' '대환대출' '사업자금' '생활비' '자동차구입' '전월세보증금' '주택구입' '투자']


In [7]:
dataset, test_df = k_fold_result(full_data)

train = dataset.reset_index().iloc[:, 1:]
train["is_applied"] = train["is_applied"].astype(int)


val_idx_0 = list(train[train['fold'] == 0].index)
val_df_0 = train[train['fold'] == 0].reset_index(drop=True).iloc[:, :-1]
train_df_0 = train[train['fold'] != 0].reset_index(drop=True).iloc[:, :-1]

val_idx_1 = list(train[train['fold'] == 1].index)
val_df_1 = train[train['fold'] == 1].reset_index(drop=True).iloc[:, :-1]
train_df_1 = train[train['fold'] != 1].reset_index(drop=True).iloc[:, :-1]

val_idx_2 = list(train[train['fold'] == 2].index)
val_df_2 = train[train['fold'] == 2].reset_index(drop=True).iloc[:, :-1]
train_df_2 = train[train['fold'] != 2].reset_index(drop=True).iloc[:, :-1]

val_idx_3 = list(train[train['fold'] == 3].index)
val_df_3 = train[train['fold'] == 3].reset_index(drop=True).iloc[:, :-1]
train_df_3 = train[train['fold'] != 3].reset_index(drop=True).iloc[:, :-1]

val_idx_4 = list(train[train['fold'] == 4].index)
val_df_4 = train[train['fold'] == 4].reset_index(drop=True).iloc[:, :-1]
train_df_4 = train[train['fold'] != 4].reset_index(drop=True).iloc[:, :-1]


In [8]:
reg_01 = TabularPredictor.load("model_test_all/model_all_fold_0")
reg_02 = TabularPredictor.load("model_test_all/model_all_fold_1")
reg_03 = TabularPredictor.load("model_test_all/model_all_fold_2")
reg_04 = TabularPredictor.load("model_test_all/model_all_fold_3")
reg_05 = TabularPredictor.load("model_test_all/model_all_fold_4")

### Feature Importance & Save Submit file

In [9]:
submit_test = pd.read_csv("./DL_dataset/test.csv")

In [10]:
rag_01_model_p  = reg_01.predict_proba(submit_test, model = "WeightedEnsemble_L2" )
rag_02_model_p  = reg_02.predict_proba(submit_test, model = "WeightedEnsemble_L2" )
rag_03_model_p  = reg_03.predict_proba(submit_test, model = "WeightedEnsemble_L2" )
rag_04_model_p  = reg_04.predict_proba(submit_test, model = "WeightedEnsemble_L2" )
rag_05_model_p  = reg_05.predict_proba(submit_test, model = "WeightedEnsemble_L2" )

/opt/conda/envs/bigcon/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [11]:
non_tau = (rag_01_model_p + rag_02_model_p + rag_03_model_p + rag_04_model_p + rag_05_model_p)
non_tau["pred"] = (non_tau.iloc[:, 0] <non_tau.iloc[:, 1]).astype(int)

In [12]:
submit_test_imp_0 = submit_test.copy()

In [13]:
submit_test_imp_0["is_applied"] =non_tau["pred"] 

In [14]:
submit_test_imp_0_ = reg_01.feature_importance(submit_test_imp_0)
submit_test_imp_1_ = reg_02.feature_importance(submit_test_imp_0)
submit_test_imp_2_ = reg_03.feature_importance(submit_test_imp_0)
submit_test_imp_3_ = reg_04.feature_importance(submit_test_imp_0)
submit_test_imp_4_ = reg_05.feature_importance(submit_test_imp_0)

In [302]:
submit_test_imp_0_.head(15)

,importance,stddev,p_value,n,p99_high,p99_low
loan_rate/min_loan_rate,0.663509,0.018806,7.736291e-08,5,0.702231,0.624787
loan_limit/max_loan_limit,0.268196,0.022976,6.400664e-06,5,0.315504,0.220889
product_id,0.220904,0.014717,2.349982e-06,5,0.251206,0.190602
loan_rate,0.153597,0.010154,2.278430e-06,5,0.174504,0.132690
loan_limit,0.133754,0.017773,3.654453e-05,5,0.170348,0.097159
desired_amount,0.111156,0.013631,2.659988e-05,5,0.139222,0.083090
yearly_income,0.043845,0.006704,6.358268e-05,5,0.057648,0.030042
bank_id,0.043399,0.010459,3.753072e-04,5,0.064935,0.021863
credit_score,0.033447,0.008269,4.138849e-04,5,0.050472,0.016422
existing_loan_amt,0.029543,0.011952,2.617374e-03,5,0.054153,0.004933


In [303]:
submit_test_imp_1_.head(15)

,importance,stddev,p_value,n,p99_high,p99_low
loan_rate/min_loan_rate,0.643367,0.024255,2.419494e-07,5,0.693308,0.593425
loan_limit/max_loan_limit,0.258457,0.025694,1.156835e-05,5,0.311361,0.205552
loan_rate,0.216391,0.009014,3.604845e-07,5,0.234951,0.197831
product_id,0.207680,0.018101,6.855972e-06,5,0.244952,0.170409
bank_id,0.125104,0.023232,1.363738e-04,5,0.172939,0.077269
loan_limit,0.106391,0.015165,4.821661e-05,5,0.137615,0.075167
desired_amount,0.091844,0.021049,3.090918e-04,5,0.135185,0.048504
yearly_income,0.034655,0.004994,5.035561e-05,5,0.044938,0.024372
credit_score,0.027312,0.007840,7.324598e-04,5,0.043455,0.011169
existing_loan_amt,0.026326,0.007454,6.951365e-04,5,0.041673,0.010979


In [304]:
submit_test_imp_2_.head(15)

,importance,stddev,p_value,n,p99_high,p99_low
loan_rate/min_loan_rate,0.644074,0.022948,1.930488e-07,5,0.691324,0.596824
loan_limit/max_loan_limit,0.260087,0.013262,8.083176e-07,5,0.287393,0.232782
product_id,0.223166,0.010872,6.736874e-07,5,0.245550,0.200781
loan_rate,0.176885,0.008127,5.333686e-07,5,0.193619,0.160150
loan_limit,0.122031,0.026752,2.602688e-04,5,0.177114,0.066947
desired_amount,0.101008,0.016755,8.761940e-05,5,0.135508,0.066509
bank_id,0.050824,0.012920,4.606689e-04,5,0.077426,0.024222
yearly_income,0.037523,0.008790,3.363196e-04,5,0.055621,0.019425
credit_score,0.030117,0.006067,1.873306e-04,5,0.042608,0.017625
existing_loan_amt,0.029518,0.010538,1.657456e-03,5,0.051216,0.007820


In [305]:
submit_test_imp_3_.head(15)

,importance,stddev,p_value,n,p99_high,p99_low
loan_rate/min_loan_rate,0.649157,0.026124,3.140424e-07,5,0.702946,0.595367
loan_limit/max_loan_limit,0.270703,0.022433,5.607577e-06,5,0.316892,0.224514
product_id,0.251593,0.017908,3.059732e-06,5,0.288466,0.214719
loan_rate,0.167181,0.012486,3.706413e-06,5,0.192890,0.141471
loan_limit,0.133873,0.015920,2.354992e-05,5,0.166652,0.101094
desired_amount,0.102318,0.017018,8.855067e-05,5,0.137360,0.067277
bank_id,0.065948,0.010175,6.588982e-05,5,0.086898,0.044998
yearly_income,0.037781,0.009505,4.426614e-04,5,0.057352,0.018211
existing_loan_amt,0.033337,0.007612,3.047614e-04,5,0.049010,0.017663
credit_score,0.029556,0.007005,3.519119e-04,5,0.043979,0.015132


In [306]:
submit_test_imp_4_.head(15)

,importance,stddev,p_value,n,p99_high,p99_low
loan_rate/min_loan_rate,0.651635,0.026989,3.522793e-07,5,0.707205,0.596065
loan_limit/max_loan_limit,0.254751,0.016200,1.951879e-06,5,0.288107,0.221395
product_id,0.254326,0.028772,1.932391e-05,5,0.313567,0.195085
loan_rate,0.129410,0.013138,1.257526e-05,5,0.156462,0.102358
loan_limit,0.125588,0.026100,2.115306e-04,5,0.179329,0.071847
desired_amount,0.100908,0.020422,1.907763e-04,5,0.142957,0.058859
yearly_income,0.043690,0.007106,8.109789e-05,5,0.058321,0.029058
bank_id,0.034409,0.010436,9.020432e-04,5,0.055897,0.012920
credit_score,0.031385,0.007729,4.078738e-04,5,0.047300,0.015470
existing_loan_amt,0.026539,0.008630,1.171743e-03,5,0.044309,0.008770


## Case Study

In [16]:
submit_test_origin = pd.read_csv("./data/loan_result.csv")
user_spec = pd.read_csv("./data/user_spec.csv")
submit_test_origin = submit_test_origin.merge(user_spec)
submit_test_origin = submit_test_origin[submit_test_origin["loanapply_insert_time"] >= "2022-06-01"]

submit_test_origin = submit_test_origin.reset_index().iloc[:, 1:]
submit_test_origin["is_applied"] = non_tau["pred"]


In [17]:
case_study = submit_test_origin.copy()

In [18]:
case_study = case_study[["application_id", "loan_limit", "loan_rate", "is_applied", "user_id"]]

In [19]:
case_study[case_study["is_applied"] >0 ]["user_id"].unique()
print(len(case_study[case_study["is_applied"] >0 ]["user_id"].unique()))

96296


In [20]:
case_study[case_study["is_applied"] >0 ]["application_id"].unique()
print(len(case_study[case_study["is_applied"] >0 ]["application_id"].unique()))

185830


In [21]:
@ray.remote
def case_study_df(data):
    unique_application_id = data[data["is_applied"] >0 ]["application_id"].unique()
    l = []
    top_5 = []
    top_10 = []
    
    for i in unique_application_id:
        check_module = data[data["application_id"] == i].copy()
        
        check_module["eff"] = check_module["loan_limit"] / check_module["loan_rate"] 
        
        num_is_applied =  check_module["is_applied"].sum()   # Input num(is_applied)
        loan_rate_top_k = check_module.sort_values("loan_rate").iloc[:num_is_applied, 3]
        loan_rate_percentage = loan_rate_top_k.sum()
        
        
        top_5.append( (check_module.sort_values("eff", ascending=False).iloc[:5, 3].sum() > 0 ).astype(int) ) 
        top_10.append( (check_module.sort_values("eff", ascending=False).iloc[:10, 3].sum() > 0 ).astype(int) ) 
        
        l.append(loan_rate_percentage)

    return l, top_5, top_10


# loan_rate  top K, Where K =  num(is_applied) + 1


In [22]:
case_study_put = ray.put(case_study)

In [23]:
result0 =  case_study_df.remote(case_study_put)

In [24]:
zz0, zz5, zz10 = ray.get(result0)

In [36]:
np.sum(zz5) / len(zz5)

0.9217187752246677

In [37]:
np.sum(zz10) / len(zz10)

0.9638217725878491

In [47]:
(pd.DataFrame(zz0) >= 1).astype(int).sum() / len(zz0)

0    0.8976
dtype: float64

In [90]:
case_study["is_applied"].value_counts()

0    3039664
1     217575
Name: is_applied, dtype: int64

### With DL models

In [48]:
test_0 = pd.read_csv("./DL_dataset/fold_0_test_df.csv")
test_1 = pd.read_csv("./DL_dataset/fold_1_test_df.csv")
test_2 = pd.read_csv("./DL_dataset/fold_2_test_df.csv")
test_3 = pd.read_csv("./DL_dataset/fold_3_test_df.csv")
test_4 = pd.read_csv("./DL_dataset/fold_4_test_df.csv")
test = (test_0 + test_1 + test_2 + test_3 + test_4) / 5

k = (test["value_1"] >= test["value_0"]).astype(int)
test["pred"] = k

In [50]:
test

,0,1,2,3,4,5,6,7,8,9,...,369,370,371,372,373,374,375,value_0,value_1,pred
0,0.003113,0.045926,0.012243,14.303822,-0.027388,-0.033889,-0.019412,6.903150,-0.018311,0.008804,...,0.009821,-0.003049,-0.007748,0.027120,0.009959,0.008927,-0.007076,1.995371,1.006201,0
1,0.011798,0.029826,-0.000721,10.385849,-0.013157,-0.007338,-0.012734,4.486781,-0.015898,0.012957,...,0.006516,-0.005503,0.005508,0.012482,-0.001722,0.004166,-0.007041,1.315746,0.855577,0
2,-0.002713,0.024109,0.009721,12.535582,-0.017334,-0.027054,-0.008413,4.913435,-0.014055,0.015022,...,0.009607,-0.007836,-0.006875,0.014437,0.009485,0.005807,-0.015015,1.714677,0.977592,0
3,0.008795,0.030560,0.022173,-6.675273,-0.028543,-0.026038,-0.029509,-3.183170,-0.007634,0.010467,...,-0.019370,-0.005606,0.012967,0.020217,0.025078,-0.007252,-0.031651,-1.437587,0.316877,1
4,-0.002451,-0.002140,0.012477,29.862645,-0.005866,-0.005563,-0.038997,13.171919,-0.006223,0.008194,...,-0.010590,-0.014812,-0.020492,-0.001840,-0.014858,-0.011623,-0.023048,4.011075,2.001666,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3257234,0.033680,0.012902,0.019410,26.456500,0.019267,0.018257,0.001824,11.818616,0.033129,0.027123,...,-0.002224,-0.023741,-0.010329,0.007774,-0.019741,-0.001132,-0.005654,3.037618,2.252133,0
3257235,0.029995,0.017237,0.016425,12.575663,0.004344,-0.015580,-0.015470,5.777054,0.012647,0.023804,...,-0.017405,-0.004163,0.010074,0.000593,0.011841,0.033838,-0.008726,1.679385,0.781392,0
3257236,0.037843,0.007100,0.019570,34.501960,-0.013508,-0.008720,-0.002568,15.427505,0.004410,0.019740,...,-0.015776,-0.007132,-0.001956,0.003109,0.003980,0.020484,-0.018957,5.449250,1.724104,0
3257237,0.039376,0.010231,0.008232,20.655818,0.001827,-0.010331,-0.005791,9.240522,0.022982,0.009280,...,-0.003019,-0.001360,0.013123,-0.004344,-0.008494,0.008374,-0.000057,3.053662,1.236984,0


In [260]:
voted_result =  ((rag_01_model_p.iloc[:, 0] <= rag_01_model_p.iloc[:, 1]).astype(int) * 1.1) +\
                ((rag_02_model_p.iloc[:, 0] <= rag_02_model_p.iloc[:, 1]).astype(int) * 1.1) +\
                ((rag_03_model_p.iloc[:, 0] <= rag_03_model_p.iloc[:, 1]).astype(int) * 1.1) +\
                ((rag_04_model_p.iloc[:, 0] <= rag_04_model_p.iloc[:, 1]).astype(int) * 1.1) +\
                ((rag_05_model_p.iloc[:, 0] <= rag_05_model_p.iloc[:, 1]).astype(int) * 1.1) +\
                test["pred"]


In [261]:
voted_result = pd.DataFrame( {"vote" : voted_result} )

In [262]:
(voted_result["vote"] > 4.5).astype(int).value_counts()

0    3063878
1     193361
Name: vote, dtype: int64

In [263]:
(voted_result["vote"] >= 3.6).astype(int).value_counts()

0    3047551
1     209688
Name: vote, dtype: int64

In [264]:
193361/ (3063878 + 193361)

0.059363467034503764

In [265]:
voted_result["vote_5_0"] = (voted_result["vote"] >= 5.0).astype(int)
voted_result["vote_4_8"] = (voted_result["vote"] >= 4.8).astype(int)
voted_result["vote_4_6"] = (voted_result["vote"] >= 4.6).astype(int)
voted_result["vote_4_4"] = (voted_result["vote"] >= 4.4).astype(int)
voted_result["vote_4_2"] = (voted_result["vote"] >= 4.2).astype(int)

voted_result["vote_4_0"] = (voted_result["vote"] >= 4.0).astype(int)
voted_result["vote_3_9"] = (voted_result["vote"] >= 3.9).astype(int)
voted_result["vote_3_8"] = (voted_result["vote"] >= 3.8).astype(int)
voted_result["vote_3_7"] = (voted_result["vote"] >= 3.7).astype(int)
voted_result["vote_3_6"] = (voted_result["vote"] >= 3.6).astype(int)
voted_result["vote_3_5"] = (voted_result["vote"] >= 3.5).astype(int)
voted_result["vote_3_4"] = (voted_result["vote"] >= 3.4).astype(int)
voted_result["vote_3_3"] = (voted_result["vote"] >= 3.3).astype(int)
voted_result["vote_3_2"] = (voted_result["vote"] >= 3.2).astype(int)
voted_result["vote_3_1"] = (voted_result["vote"] >= 3.1).astype(int)
voted_result["vote_3_0"] = (voted_result["vote"] >= 3.0).astype(int)

In [266]:
case_study_3_0 = case_study.copy()
case_study_3_1 = case_study.copy()
case_study_3_2 = case_study.copy()
case_study_3_3 = case_study.copy()
case_study_3_4 = case_study.copy()
case_study_3_5 = case_study.copy()
case_study_3_6 = case_study.copy()
case_study_3_7 = case_study.copy()
case_study_3_8 = case_study.copy()
case_study_3_9 = case_study.copy()
case_study_4_0 = case_study.copy()
case_study_4_2 = case_study.copy()
case_study_4_4 = case_study.copy()
case_study_4_6 = case_study.copy()
case_study_4_8 = case_study.copy()
case_study_5_0 = case_study.copy()

In [267]:
case_study_3_0["is_applied"] = voted_result["vote_3_0"]
case_study_3_1["is_applied"] = voted_result["vote_3_1"]
case_study_3_2["is_applied"] = voted_result["vote_3_2"]
case_study_3_3["is_applied"] = voted_result["vote_3_3"]
case_study_3_4["is_applied"] = voted_result["vote_3_4"]
case_study_3_5["is_applied"] = voted_result["vote_3_5"]
case_study_3_6["is_applied"] = voted_result["vote_3_6"]
case_study_3_7["is_applied"] = voted_result["vote_3_7"]
case_study_3_8["is_applied"] = voted_result["vote_3_8"]
case_study_3_9["is_applied"] = voted_result["vote_3_9"]
case_study_4_0["is_applied"] = voted_result["vote_4_0"]

case_study_4_2["is_applied"] = voted_result["vote_4_2"]
case_study_4_4["is_applied"] = voted_result["vote_4_4"]
case_study_4_6["is_applied"] = voted_result["vote_4_6"]
case_study_4_8["is_applied"] = voted_result["vote_4_8"]
case_study_5_0["is_applied"] = voted_result["vote_5_0"]

In [269]:
case_study_put_3_0 = ray.put(case_study_3_0)
case_study_put_3_1 = ray.put(case_study_3_1)
case_study_put_3_2 = ray.put(case_study_3_2)
case_study_put_3_3 = ray.put(case_study_3_3)
case_study_put_3_4 = ray.put(case_study_3_4)
case_study_put_3_5 = ray.put(case_study_3_5)
case_study_put_3_6 = ray.put(case_study_3_6)
case_study_put_3_7 = ray.put(case_study_3_7)
case_study_put_3_8 = ray.put(case_study_3_8)
case_study_put_3_9 = ray.put(case_study_3_9)
case_study_put_4_0 = ray.put(case_study_4_0)
case_study_put_4_2 = ray.put(case_study_4_2)
case_study_put_4_4 = ray.put(case_study_4_4)
case_study_put_4_6 = ray.put(case_study_4_6)
case_study_put_4_8 = ray.put(case_study_4_8)
case_study_put_5_0 = ray.put(case_study_5_0)

In [270]:
result_3_0 =  case_study_df.remote(case_study_put_3_0)
result_3_1 =  case_study_df.remote(case_study_put_3_1)
result_3_2 =  case_study_df.remote(case_study_put_3_2)
result_3_3 =  case_study_df.remote(case_study_put_3_3)
result_3_4 =  case_study_df.remote(case_study_put_3_4)
result_3_5 =  case_study_df.remote(case_study_put_3_5)
result_3_6 =  case_study_df.remote(case_study_put_3_6)
result_3_7 =  case_study_df.remote(case_study_put_3_7)
result_3_8 =  case_study_df.remote(case_study_put_3_8)
result_3_9 =  case_study_df.remote(case_study_put_3_9)
result_4_0 =  case_study_df.remote(case_study_put_4_0)

result_4_2 =  case_study_df.remote(case_study_put_4_2)
result_4_4 =  case_study_df.remote(case_study_put_4_4)
result_4_6 =  case_study_df.remote(case_study_put_4_6)
result_4_8 =  case_study_df.remote(case_study_put_4_8)
result_5_0 =  case_study_df.remote(case_study_put_5_0)

In [271]:
zz0_3_0, zz5_3_0, zz10_3_0 = ray.get(result_3_0)
zz0_3_1, zz5_3_1, zz10_3_1 = ray.get(result_3_1)
zz0_3_2, zz5_3_2, zz10_3_2 = ray.get(result_3_2)
zz0_3_3, zz5_3_3, zz10_3_3 = ray.get(result_3_3)
zz0_3_4, zz5_3_4, zz10_3_4 = ray.get(result_3_4)
zz0_3_5, zz5_3_5, zz10_3_5 = ray.get(result_3_5)
zz0_3_6, zz5_3_6, zz10_3_6 = ray.get(result_3_6)
zz0_3_7, zz5_3_7, zz10_3_7 = ray.get(result_3_7)
zz0_3_8, zz5_3_8, zz10_3_8 = ray.get(result_3_8)
zz0_3_9, zz5_3_9, zz10_3_9 = ray.get(result_3_9)
zz0_4_0, zz5_4_0, zz10_4_0 = ray.get(result_4_0)

zz0_4_2, zz5_4_2, zz10_4_2 = ray.get(result_4_2)
zz0_4_4, zz5_4_4, zz10_4_4 = ray.get(result_4_4)
zz0_4_6, zz5_4_6, zz10_4_6 = ray.get(result_4_6)
zz0_4_8, zz5_4_8, zz10_4_8 = ray.get(result_4_8)
zz0_5_0, zz5_5_0, zz10_5_0 = ray.get(result_5_0)

In [272]:
print("3_0", (np.sum(zz5_3_0) / len(zz5_3_0)) * 100 )
print("3_1", (np.sum(zz5_3_1) / len(zz5_3_1)) * 100 )
print("3_2", (np.sum(zz5_3_2) / len(zz5_3_2)) * 100 )
print("3_3", (np.sum(zz5_3_3) / len(zz5_3_3)) * 100 )
print("3_4", (np.sum(zz5_3_4) / len(zz5_3_4)) * 100 )
print("3_5", (np.sum(zz5_3_5) / len(zz5_3_5)) * 100 )
print("3_6", (np.sum(zz5_3_6) / len(zz5_3_6)) * 100 )
print("3_7", (np.sum(zz5_3_7) / len(zz5_3_7)) * 100 )
print("3_8", (np.sum(zz5_3_8) / len(zz5_3_8)) * 100 )
print("3_9", (np.sum(zz5_3_9) / len(zz5_3_9)) * 100 )
print("4_0", (np.sum(zz5_4_0) / len(zz5_4_0)) * 100 )

print("4_2", (np.sum(zz5_4_2) / len(zz5_4_2)) * 100 )
print("4_4", (np.sum(zz5_4_4) / len(zz5_4_4)) * 100 )
print("4_6", (np.sum(zz5_4_6) / len(zz5_4_6)) * 100 )
print("4_8", (np.sum(zz5_4_8) / len(zz5_4_8)) * 100 )
print("5_0", (np.sum(zz5_5_0) / len(zz5_5_0)) * 100 )

3_0 92.07939236841125
3_1 92.07939236841125
3_2 92.07939236841125
3_3 92.22116978451338
3_4 92.366197027399
3_5 92.366197027399
3_6 92.366197027399
3_7 92.366197027399
3_8 92.366197027399
3_9 92.366197027399
4_0 92.366197027399
4_2 92.366197027399
4_4 92.50071381783368
4_6 92.64042520951593
4_8 92.64042520951593
5_0 92.64042520951593


In [273]:
print("3_0", (np.sum(zz10_3_0) / len(zz10_3_0)) * 100 )
print("3_1", (np.sum(zz10_3_1) / len(zz10_3_1)) * 100 )
print("3_2", (np.sum(zz10_3_2) / len(zz10_3_2)) * 100 )
print("3_3", (np.sum(zz10_3_3) / len(zz10_3_3)) * 100 )
print("3_4", (np.sum(zz10_3_4) / len(zz10_3_4)) * 100 )
print("3_5", (np.sum(zz10_3_5) / len(zz10_3_5)) * 100 )
print("3_6", (np.sum(zz10_3_6) / len(zz10_3_6)) * 100 )
print("3_7", (np.sum(zz10_3_7) / len(zz10_3_7)) * 100 )
print("3_8", (np.sum(zz10_3_8) / len(zz10_3_8)) * 100 )
print("3_9", (np.sum(zz10_3_9) / len(zz10_3_9)) * 100 )
print("4_0", (np.sum(zz10_4_0) / len(zz10_4_0)) * 100 )

print("4_2", (np.sum(zz10_4_2) / len(zz10_4_2)) * 100 )
print("4_4", (np.sum(zz10_4_4) / len(zz10_4_4)) * 100 )
print("4_6", (np.sum(zz10_4_6) / len(zz10_4_6)) * 100 )
print("4_8", (np.sum(zz10_4_8) / len(zz10_4_8)) * 100 )
print("5_0", (np.sum(zz10_5_0) / len(zz10_5_0)) * 100 )

3_0 96.33270703705084
3_1 96.33270703705084
3_2 96.33270703705084
3_3 96.38882311153209
3_4 96.51722594089067
3_5 96.51722594089067
3_6 96.51722594089067
3_7 96.51722594089067
3_8 96.51722594089067
3_9 96.51722594089067
4_0 96.51722594089067
4_2 96.51722594089067
4_4 96.574234255418
4_6 96.70427657268075
4_8 96.70427657268075
5_0 96.70427657268075


In [274]:
print("3_0", (pd.DataFrame(zz0_3_0) >= 1).astype(int).sum() / len(zz0_3_0))
print("3_1", (pd.DataFrame(zz0_3_1) >= 1).astype(int).sum() / len(zz0_3_1))
print("3_2", (pd.DataFrame(zz0_3_2) >= 1).astype(int).sum() / len(zz0_3_2))
print("3_3", (pd.DataFrame(zz0_3_3) >= 1).astype(int).sum() / len(zz0_3_3))
print("3_4", (pd.DataFrame(zz0_3_4) >= 1).astype(int).sum() / len(zz0_3_4))
print("3_5", (pd.DataFrame(zz0_3_5) >= 1).astype(int).sum() / len(zz0_3_5))
print("3_6", (pd.DataFrame(zz0_3_6) >= 1).astype(int).sum() / len(zz0_3_6))
print("3_7", (pd.DataFrame(zz0_3_7) >= 1).astype(int).sum() / len(zz0_3_7))
print("3_8", (pd.DataFrame(zz0_3_8) >= 1).astype(int).sum() / len(zz0_3_8))
print("3_9", (pd.DataFrame(zz0_3_9) >= 1).astype(int).sum() / len(zz0_3_9))
print("4_0", (pd.DataFrame(zz0_4_0) >= 1).astype(int).sum() / len(zz0_4_0))

print("4_2", (pd.DataFrame(zz0_4_2) >= 1).astype(int).sum() / len(zz0_4_2))
print("4_4", (pd.DataFrame(zz0_4_4) >= 1).astype(int).sum() / len(zz0_4_4))
print("4_6", (pd.DataFrame(zz0_4_6) >= 1).astype(int).sum() / len(zz0_4_6))
print("4_8", (pd.DataFrame(zz0_4_8) >= 1).astype(int).sum() / len(zz0_4_8))
print("5_0", (pd.DataFrame(zz0_5_0) >= 1).astype(int).sum() / len(zz0_5_0))


3_0 0    0.89722
dtype: float64
3_1 0    0.89722
dtype: float64
3_2 0    0.89722
dtype: float64
3_3 0    0.897181
dtype: float64
3_4 0    0.899419
dtype: float64
3_5 0    0.899419
dtype: float64
3_6 0    0.899419
dtype: float64
3_7 0    0.899419
dtype: float64
3_8 0    0.899419
dtype: float64
3_9 0    0.899419
dtype: float64
4_0 0    0.899419
dtype: float64
4_2 0    0.899419
dtype: float64
4_4 0    0.899825
dtype: float64
4_6 0    0.902422
dtype: float64
4_8 0    0.902422
dtype: float64
5_0 0    0.902422
dtype: float64


## Voting result

In [275]:
voted_result =  ((rag_01_model_p.iloc[:, 0] <= rag_01_model_p.iloc[:, 1]).astype(int) * 1.1) +\
                ((rag_02_model_p.iloc[:, 0] <= rag_02_model_p.iloc[:, 1]).astype(int) * 1.1) +\
                ((rag_03_model_p.iloc[:, 0] <= rag_03_model_p.iloc[:, 1]).astype(int) * 1.1) +\
                ((rag_04_model_p.iloc[:, 0] <= rag_04_model_p.iloc[:, 1]).astype(int) * 1.1) +\
                ((rag_05_model_p.iloc[:, 0] <= rag_05_model_p.iloc[:, 1]).astype(int) * 1.1) +\
                test["pred"]

In [276]:
voted_result = pd.DataFrame( {"vote" : voted_result} )

In [279]:
(voted_result["vote"] >= 4.6).astype(int).value_counts()

0    3063878
1     193361
Name: vote, dtype: int64

0          0
1          0
2          0
3          1
4          0
          ..
3257234    0
3257235    0
3257236    0
3257237    0
3257238    0
Name: vote, Length: 3257239, dtype: int64

In [280]:
submit_dataset = submit_test_origin.copy()

In [283]:
submit_dataset["is_applied"] = (voted_result["vote"] >= 4.6).astype(int)

In [287]:
submit_dataset[["application_id","product_id","is_applied"]].to_csv("./submit/test.csv", index = False)

In [288]:
submit_dataset

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,...,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,0,430982,1996.0,1.0,...,EARNEDINCOME,20220201.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,0,430982,1996.0,1.0,...,EARNEDINCOME,20220201.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,0,430982,1996.0,1.0,...,EARNEDINCOME,20220201.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,1,430982,1996.0,1.0,...,EARNEDINCOME,20220201.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,0,430982,1996.0,1.0,...,EARNEDINCOME,20220201.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3257234,1428218,2022-06-03 12:01:58,62,200,3000000.0,14.8,0,478802,1965.0,1.0,...,EARNEDINCOME,20190301.0,정규직,기타가족소유,50000000.0,대환대출,0.0,NaN,2.0,56000000.0
3257235,1428218,2022-06-03 12:01:49,2,7,40000000.0,11.8,0,478802,1965.0,1.0,...,EARNEDINCOME,20190301.0,정규직,기타가족소유,50000000.0,대환대출,0.0,NaN,2.0,56000000.0
3257236,1428218,2022-06-03 12:01:48,32,257,15000000.0,7.2,0,478802,1965.0,1.0,...,EARNEDINCOME,20190301.0,정규직,기타가족소유,50000000.0,대환대출,0.0,NaN,2.0,56000000.0
3257237,1428218,2022-06-03 12:01:48,33,110,44000000.0,13.5,0,478802,1965.0,1.0,...,EARNEDINCOME,20190301.0,정규직,기타가족소유,50000000.0,대환대출,0.0,NaN,2.0,56000000.0


#### Feature Importance DL

In [308]:
submit_test_imp_0["is_applied"] = (voted_result["vote"] >= 4.6).astype(int)

In [309]:
submit_test_imp_0_DL = reg_01.feature_importance(submit_test_imp_0)
submit_test_imp_1_DL = reg_02.feature_importance(submit_test_imp_0)
submit_test_imp_2_DL = reg_03.feature_importance(submit_test_imp_0)
submit_test_imp_3_DL = reg_04.feature_importance(submit_test_imp_0)
submit_test_imp_4_DL = reg_05.feature_importance(submit_test_imp_0)

In [310]:
submit_test_imp_0_DL

,importance,stddev,p_value,n,p99_high,p99_low
loan_rate/min_loan_rate,0.640639,0.025392,2.955289e-07,5,0.692921,0.588356
loan_limit/max_loan_limit,0.251654,0.020848,5.601045e-06,5,0.294581,0.208728
product_id,0.220413,0.015557,2.958583e-06,5,0.252446,0.188381
loan_rate,0.153739,0.010755,2.855117e-06,5,0.175884,0.131595
loan_limit,0.136343,0.017524,3.203562e-05,5,0.172424,0.100262
desired_amount,0.102438,0.022613,2.673591e-04,5,0.148999,0.055877
yearly_income,0.046537,0.008231,1.127062e-04,5,0.063485,0.029589
bank_id,0.043074,0.011760,6.053328e-04,5,0.067289,0.018860
existing_loan_amt,0.027951,0.016680,1.000033e-02,5,0.062296,-0.006394
credit_score,0.026915,0.011031,2.743099e-03,5,0.049629,0.004201


In [311]:
submit_test_imp_1_DL

,importance,stddev,p_value,n,p99_high,p99_low
loan_rate/min_loan_rate,0.623094,0.035572,1.269177e-06,5,0.696338,0.549851
loan_limit/max_loan_limit,0.243241,0.019347,4.762281e-06,5,0.283076,0.203406
loan_rate,0.214994,0.009264,4.126648e-07,5,0.234069,0.195919
product_id,0.209497,0.019296,8.538817e-06,5,0.249227,0.169767
bank_id,0.119351,0.021296,1.166476e-04,5,0.163200,0.075501
loan_limit,0.110345,0.018125,8.429895e-05,5,0.147665,0.073026
desired_amount,0.084779,0.027383,1.142409e-03,5,0.141160,0.028397
yearly_income,0.037014,0.008257,2.783912e-04,5,0.054014,0.020013
existing_loan_amt,0.023901,0.007699,1.130787e-03,5,0.039753,0.008049
credit_score,0.020004,0.008387,2.975959e-03,5,0.037273,0.002735


In [ ]:
submit_test_imp_2_DL

In [312]:
submit_test_imp_3_DL

,importance,stddev,p_value,n,p99_high,p99_low
loan_rate/min_loan_rate,0.631825,0.029679,5.825430e-07,5,0.692935,0.570715
loan_limit/max_loan_limit,0.256595,0.021061,5.397973e-06,5,0.299960,0.213229
product_id,0.252856,0.020451,5.090236e-06,5,0.294964,0.210748
loan_rate,0.166591,0.015419,8.707223e-06,5,0.198339,0.134843
loan_limit,0.135669,0.019596,5.080917e-05,5,0.176018,0.095321
desired_amount,0.094420,0.028058,8.350243e-04,5,0.152192,0.036647
bank_id,0.067230,0.018118,5.760825e-04,5,0.104535,0.029924
yearly_income,0.044952,0.011319,4.442369e-04,5,0.068259,0.021645
existing_loan_amt,0.028801,0.012224,3.109175e-03,5,0.053970,0.003632
desired_amount/existing_loan_amt,0.024982,0.004504,1.215042e-04,5,0.034257,0.015708


(case_study_df pid=22929) E1008 12:46:13.254708924   23258 chttp2_transport.cc:2887]   keepalive_ping_end state error: 0 (expect: 1)
(pid=gcs_server) E1008 12:39:26.790332506   22795 chttp2_transport.cc:2846]   keepalive_ping_end state error: 0 (expect: 1)
(pid=gcs_server) [2022-10-08 12:46:27,340 E 22779 22779] (gcs_server) gcs_server.cc:283: Failed to get the resource load: GrpcUnavailable: RPC Error message: Socket closed; RPC Error details: 
(pid=gcs_server) [2022-10-08 12:46:43,638 E 22779 22779] (gcs_server) gcs_server.cc:283: Failed to get the resource load: GrpcUnavailable: RPC Error message: failed to connect to all addresses; RPC Error details: 
(pid=22923) E1008 12:46:13.254724047   23158 chttp2_transport.cc:2887]   keepalive_ping_end state error: 0 (expect: 1)
(pid=22918) E1008 12:46:13.254738837   23134 chttp2_transport.cc:2887]   keepalive_ping_end state error: 0 (expect: 1)
(pid=gcs_server) [2022-10-08 12:47:14,764 E 22779 22779] (gcs_server) gcs_server.cc:283: Failed to

In [1]:
import pandas as pd